# Lab 4
## Arc I
### Mohsen Ahmadkhani

In [83]:
# Download temperature data from NDAWN
import requests
from urllib.parse import urlencode
import pandas as pd 
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import folium
import time

def get_weather(start_date, end_date, station_count = None):
    df = []
    row = 1
    for station in range(1, station_count):
        base_url = 'https://ndawn.ndsu.nodak.edu/table.csv?'
        params = f"station={str(station)}&begin_date={start_date}&end_date={end_date}&ttype=daily&quick_pick=&variable=ddmxt&variable=ddmnt&variable=ddavt&variable=ddbst&variable=ddtst&variable=ddws&variable=ddmxws&variable=ddsr&variable=ddr&variable=dddp&variable=ddwc"
        url = base_url+params
        r = requests.get(url)

        with open("response.csv", "w") as text_file:
            text_file.write(r.content.decode('utf-8'))

        if row == 1:
            instance = pd.read_csv('response.csv', header=[0,1],skiprows=3)
            if instance.iloc[0][0].find('<') == -1: 
                df = instance
                row = 0
        else:
            instance = pd.read_csv('response.csv', header=[0,1],skiprows=3)
            df = pd.concat([df, instance], ignore_index=True)
    
    return df
            


today_date = list(time.localtime())
yesterday = str(today_date[0]) + '-' + str(today_date[1]) + '-' + str(today_date[2]-1)
last_month = str(today_date[0]) + '-' + str(today_date[1]-1) + '-' + str(today_date[2]-1)

start_date = last_month
end_date = yesterday

df = get_weather(start_date, end_date, 300) # get the data for all ND weather stations
df.columns = [f'{i}_{j}' for i, j in df.columns] # concat two header rows into one 

# df.to_csv('weather_monthly.csv', encoding='utf-8', index=False)

stations_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude_deg, df.Latitude_deg))
stations_gdf.crs = 4326

stations_gdf.head()




,"<title>NDAWN CSV Export Error</title>_ <meta http-equiv=""Content-Type"" content=""text/html; charset=utf-8"">",Avg Bare Soil Temp_Degrees F,Avg Bare Soil Temp Flag_Unnamed: 14_level_1,Avg Temp_Degrees F,Avg Temp Flag_Unnamed: 12_level_1,Avg Turf Soil Temp_Degrees F,Avg Turf Soil Temp Flag_Unnamed: 16_level_1,Avg Wind Speed_mph,Avg Wind Speed Flag_Unnamed: 18_level_1,Day_Unnamed: 6_level_1,...,Month_Unnamed: 5_level_1,Rainfall_inch,Rainfall Flag_Unnamed: 24_level_1,Station Name_Unnamed: 0_level_1,Total Solar Rad_Lys,Total Solar Rad Flag_Unnamed: 22_level_1,Wind Chill_Degrees F,Wind Chill Flag_Unnamed: 28_level_1,Year_Unnamed: 4_level_1,geometry
0,NaN,36.306,NaN,38.320,NaN,40.160,NaN,10.057,NaN,4,...,11,NaN,M,Eldred,230.871,NaN,30.335,NaN,2021,POINT (-96.82200 47.68800)
1,NaN,40.309,NaN,42.244,NaN,42.218,NaN,10.211,NaN,5,...,11,NaN,M,Eldred,122.051,NaN,37.222,NaN,2021,POINT (-96.82200 47.68800)
2,NaN,42.626,NaN,46.873,NaN,43.774,NaN,5.009,NaN,6,...,11,NaN,M,Eldred,182.365,NaN,41.408,NaN,2021,POINT (-96.82200 47.68800)
3,NaN,42.160,NaN,45.344,NaN,43.559,NaN,4.637,NaN,7,...,11,NaN,M,Eldred,175.323,NaN,41.385,NaN,2021,POINT (-96.82200 47.68800)
4,NaN,40.363,NaN,39.274,NaN,42.719,NaN,7.003,NaN,8,...,11,NaN,M,Eldred,156.724,NaN,36.420,NaN,2021,POINT (-96.82200 47.68800)


In [84]:
#aggregate by averaging the average daily temperature to get to the monthly average
stations_gdf = stations_gdf.groupby('Station Name_Unnamed: 0_level_1').mean() 
stations_gdf = gpd.GeoDataFrame(stations_gdf, geometry=gpd.points_from_xy(stations_gdf.Longitude_deg, stations_gdf.Latitude_deg))
stations_gdf.crs = 4326
stations_gdf = stations_gdf.reset_index()
stations_gdf


,Station Name_Unnamed: 0_level_1,Avg Bare Soil Temp_Degrees F,Avg Temp_Degrees F,Avg Temp Flag_Unnamed: 12_level_1,Avg Turf Soil Temp_Degrees F,Avg Wind Speed_mph,Dew Point_Degrees F,Latitude_deg,Longitude_deg,Max Temp_Degrees F,Max Temp Flag_Unnamed: 8_level_1,Max Wind Speed_mph,Max Wind Speed Flag_Unnamed: 20_level_1,Min Temp_Degrees F,Min Temp Flag_Unnamed: 10_level_1,Rainfall_inch,Total Solar Rad_Lys,Wind Chill_Degrees F,geometry
0,Ada,35.432065,30.250032,NaN,35.948000,8.992065,24.858129,47.321100,-96.513900,37.913548,NaN,22.956677,NaN,22.585903,NaN,NaN,129.357839,22.636968,POINT (-96.51390 47.32110)
1,Adams,35.027613,27.019419,NaN,35.537677,11.029065,22.080129,48.499814,-98.075869,35.098839,NaN,26.009032,NaN,18.939419,NaN,NaN,120.562871,17.147387,POINT (-98.07587 48.49981)
2,Alamo,32.766226,31.273387,NaN,33.396387,8.803258,22.179516,48.546510,-103.471843,41.949871,NaN,25.753258,NaN,20.596387,NaN,NaN,124.121387,24.311484,POINT (-103.47184 48.54651)
3,Alvarado,39.125194,29.391194,NaN,39.389355,10.633903,23.518645,48.245939,-97.021533,37.169645,NaN,25.871290,NaN,21.612290,NaN,NaN,116.734613,20.594645,POINT (-97.02153 48.24594)
4,Amidon,36.936677,36.960774,NaN,38.132806,9.550194,24.513742,46.488490,-103.316290,48.569968,NaN,26.307774,NaN,25.350968,NaN,NaN,155.047613,31.077710,POINT (-103.31629 46.48849)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,Williston,35.132097,34.346710,NaN,34.967935,7.898677,23.323419,48.133000,-103.739000,44.345968,NaN,23.503032,NaN,24.346903,NaN,NaN,129.717935,28.257258,POINT (-103.73900 48.13300)
137,Wishek,35.235000,33.960645,NaN,37.936935,12.407355,24.337581,46.260000,-99.666000,44.063839,NaN,30.823097,NaN,23.856903,NaN,NaN,140.713903,24.097258,POINT (-99.66600 46.26000)
138,Wolford,33.290645,29.624452,NaN,34.458645,10.039903,23.445129,48.516030,-99.624410,38.026710,NaN,25.794290,NaN,21.221710,NaN,NaN,121.023000,20.578258,POINT (-99.62441 48.51603)
139,Wolverton,35.258097,32.637226,NaN,37.566839,10.073871,25.412129,46.565453,-96.687257,41.039097,NaN,25.271871,NaN,24.234903,NaN,NaN,126.674581,24.095871,POINT (-96.68726 46.56545)


In [85]:
# plot an interactive map
import warnings
warnings.filterwarnings('ignore') 

m = folium.Map(location = [44.9779006, -93.1831530], tiles='OpenStreetMap' , zoom_start = 12) # tiles="Stamen Toner"

for _, r in stations_gdf.iterrows():

    sim_geo = gpd.GeoSeries(r['geometry']) 
    geo_j = sim_geo.to_json()
   
    geo_j = folium.GeoJson(data=geo_j, 
                           style_function = lambda x: {'color': 'red', 'weight': 1,  'fillColor': 'YlGnBu'})
    folium.Popup(f"<i>Station: {r['Station Name_Unnamed: 0_level_1']}, Average Monthly Temperature: {r['Avg Temp_Degrees F']} °F</i>").add_to(geo_j)
    folium.Tooltip(f"<i>Station: {r['Station Name_Unnamed: 0_level_1']} <br> Avg Monthly Temperature: {r['Avg Temp_Degrees F']} °F</i>").add_to(geo_j)
    geo_j.add_to(m)


sw = [44.867066, -104.248]
ne = [48.884, -93.065245]

m.fit_bounds([sw, ne]) 

m



In [86]:
# df = geopandas.GeoDataFrame(stations_gdf, geometry='geom')
# df.crs= "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs"
stations_gdf.to_file('ndawn_weather.shp', driver='ESRI Shapefile')